In [1]:
import numpy as np 
from scipy.sparse import csc_matrix, identity, diags
import sparse_dot_mkl
import scipy.sparse.linalg as linalg
from scipy import signal
import matplotlib.pyplot as plt 
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
import multiprocessing as multiproc
import time
import sys
import h5py
import WCS
import copy
import os


In [2]:
%matplotlib widget

In [29]:
class Destriper():
    def __init__(self, datapath, highcut, eps):
        
        self.infile      = infile
        self.datapath    = datapath
        #self.N_baseline  = N_baseline
        self.highcut     = highcut
        self.baseline_time = 1 / highcut
        self.eps         = eps
        
        self.Nside       = 120           # Number of pixels along RA/Dec
        self.dpix        = 2.0 / 60.0    # Pixel resolution in degrees (2' = 2/60 deg)
        self.Npix        = self.Nside ** 2   # Total number of pixels in the image
        self.fieldcent = [226, 55]  
        self.cube_filename = "/mn/stornext/d16/cmbco/comap/protodir/cube_real.npy"
        
    def run(self, nsb, nfreq):
        self.nsb         = nsb 
        self.nfreq       = nfreq 
        
        print("Loading data:")
        t0 = time.time()
        self.get_data()
        print("Loading time:", time.time() - t0, "sec")

        print("Get pixel index:")
        t0 = time.time()
        self.get_px_index()
        print("Get pixel time:", time.time() - t0, "sec")

        print("Get pointing matrix:")
        t0 = time.time()
        self.get_P()
        print("Get pointing matrix time:", time.time() - t0, "sec")

        print("Get F:")
        t0 = time.time()
        self.get_F()
        print("Get F time:", time.time() - t0, "sec")
        
        print("Get C_n_inv:")
        t0 = time.time()
        self.get_Cn_inv()
        print("Get C_n_inv time:", time.time() - t0, "sec")

        print("Get PCP_inv:")
        t0 = time.time()
        self.get_PCP_inv()
        print("Get PCP_inv time:", time.time() - t0, "sec")

        #print("Get destriped map:")
        #t0 = time.time()
        #self.get_destriped_map()
        #print("Get destriped map time:", time.time() - t0, "sec")
        
    def get_data(self):
        nsb, nfreq = self.nsb, self.nfreq
       
        tod_lens  = []
        names     = []
        Nscans    = 0
        t = time.time()
        #accpt = ["15304", "15302"] 
        accpt = ["15197", "15247", "15297"]
        #accpt = ["15197"]
        #accpt = ["15197", "15247", "15297", "15325", "15332", "15275"]
        for filename in os.listdir(self.datapath):
            if np.any([(name in filename) for name in accpt]):
                Nscans += 1
                infile = h5py.File(self.datapath + filename, "r")
                Nsamp  = infile["tod"].shape[-1]
                Nfeed  = infile["tod"].shape[0] - 1
                for i in range(Nfeed):
                    tod_lens.append(Nsamp)
                names.append(filename)
                infile.close()
        
        self.Nfeed = Nfeed
        print("Number of scans:", Nscans)
        tod_lens = np.array(tod_lens)
        tod_cumlen = np.zeros(Nscans * Nfeed + 1).astype(int)
        tod_cumlen[1:] = np.cumsum(tod_lens).astype(int)
        Nsamp_tot = np.sum(tod_lens)
    
        tod_buffer = np.zeros(Nsamp_tot)
        #time_buffer = np.zeros(Nsamp_tot)
        ra_buffer = np.zeros(Nsamp_tot)
        dec_buffer = np.zeros(Nsamp_tot)
        sigma0_buffer = np.zeros((Nscans, Nfeed))
        
        Nbaseline_tot = 0
        Nperbaselines = [0]
        Nperscan      = [0]
        
        for i in range(Nscans):
            infile = h5py.File(self.datapath + names[i], "r")
            tod    = np.array(infile["tod"])[()] #[()]#.astype(dtype=np.float32, copy=False) 
            tod[:, 0, :, :]    = tod[:, 0, ::-1, :] #[()]#.astype(dtype=np.float32, copy=False) 
            tod[:, 2, :, :]    = tod[:, 2, ::-1, :] #[()]#.astype(dtype=np.float32, copy=False) 
            tod                = tod[:-1, nsb, nfreq, :]
            
            if tod.dtype != np.float32:
                raise ValueError("The input TOD should be of dtype float32!")

            tod_time   = np.array(infile["time"])[()] * 3600 * 24
            sigma0 = np.array(infile["sigma0"])[()]
            
            sigma0[:, 0, :]    = sigma0[:, 0, ::-1] #[()]#.astype(dtype=np.float32, copy=False) 
            sigma0[:, 0, :]    = sigma0[:, 2, ::-1] #[()]#.astype(dtype=np.float32, copy=False) 
            sigma0             = sigma0[:-1, nsb, nfreq]

            pointing  = np.array(infile["point_cel"])[()] #[()]
            ra        = pointing[:-1, :, 0] 
            dec       = pointing[:-1, :, 1] 


            Nsamp = tod.shape[-1] 
            Nperscan.append(Nsamp)
            dt    = tod_time[1] - tod_time[0]
            Nperbaseline = int(round(self.baseline_time / dt))
            #N_baseline = int(round((tod_time[-1] - tod_time[0]) / self.baseline_time))
            Nbaseline = int(np.floor(Nsamp / Nperbaseline))
            #print(N_perbaseline, dt * N_perbaseline, int(round(self.baseline_time / dt)))
            excess = Nsamp - Nperbaseline * Nbaseline
            for j in range(Nfeed):
                Nbaseline_tot += Nbaseline
                
                for k in range(Nbaseline):
                    Nperbaselines.append(Nperbaseline)
            
                if excess > 0:
                    Nbaseline_tot += 1
                    Nperbaselines.append(excess)
                
                sigma0_buffer[i, j] = sigma0[j]
            
        
            start = tod_cumlen[i * Nfeed]
            end   = tod_cumlen[(i + 1) * Nfeed]

            tod_buffer[start:end]  = tod.flatten()
            #time_buffer[start:end] = tod_time
            ra_buffer[start:end]   = ra.flatten()
            dec_buffer[start:end]  = dec.flatten()
            
            infile.close()
            
        #tod_buffer = np.trim_zeros(tod_buffer)
        self.tod          = tod_buffer - np.nanmean(tod_buffer) 
        #self.time         = time_buffer      #np.trim_zeros(time_buffer)
        self.ra           = ra_buffer        #np.trim_zeros(ra_buffer)
        self.dec          = dec_buffer       #np.trim_zeros(dec_buffer)
        self.sigma0       = sigma0_buffer.flatten()
        self.Nbaseline    = Nbaseline_tot
        self.Nperbaselines = np.array(Nperbaselines)
        self.Nsamp        = Nsamp_tot
        self.Nscans       = Nscans * Nfeed
        self.tod_cumlen   = tod_cumlen
       
        """
        infile      = h5py.File(self.infile, "r")
        self.tod    = np.array(infile["tod"])[:-1, nsb, nfreq, :] #[()]#.astype(dtype=np.float32, copy=False) 
        if self.tod.dtype != np.float32:
            raise ValueError("The input TOD should be of dtype float32!")
        
        self.time   = np.array(infile["time"])[()] * 3600 * 24
        self.sigma0 = np.array(infile["sigma0"])[:-1, nsb, nfreq] #[()]

        pointing    = np.array(infile["point_cel"])[()] #[()]
        self.ra     = pointing[:-1, :, 0] 
        self.dec    = pointing[:-1, :, 1] 

        #self.tod[:, 0, :, :] = self.tod[:, 0, ::-1, :]
        #self.tod[:, 2, :, :] = self.tod[:, 2, ::-1, :]
        #self.Nfeeds, self.Nsb, self.Nfreq, self.Nsamp = self.tod.shape
        self.Nfeeds, self.Nsamp = self.tod.shape
        self.dt    = self.time[1] - self.time[0]
        self.N_baseline = int(round((self.time[-1] - self.time[0]) / self.baseline_time))
        self.N_perbaseline = int(np.floor(self.Nsamp / self.N_baseline))
        
        self.Nperfeed = self.N_perbaseline * self.N_baseline
        self.Nsamp = self.Nperfeed * self.Nfeeds
        self.N_baseline *= self.Nfeeds
        
        self.tod  = self.tod[:, :self.Nperfeed]
        self.tod  = self.tod.flatten()
        self.tod  -= np.nanmean(self.tod) 
        
        self.time = self.time[:self.Nperfeed]
        self.ra   = self.ra[:, :self.Nperfeed].flatten()
        self.dec  = self.dec[:, :self.Nperfeed].flatten()
        
        print(self.ra.shape, self.dec.shape, self.Nsamp)
        
        print(len(self.tod)) 
        print(self.N_baseline, self.time[-1] - self.time[0], self.dt * self.N_baseline, np.floor(self.Nsamp / self.N_baseline))
        infile.close()
        """

    def get_px_index(self):
        Nside, dpix, fieldcent, ra, dec = self.Nside, self.dpix, self.fieldcent, self.ra, self.dec

        self.px = WCS.ang2pix([Nside, Nside], [-dpix, dpix], fieldcent, dec, ra)
        
    def get_P(self):
        Nsamp, Npix = self.Nsamp, self.Npix

        ones = np.ones(Nsamp)
        rows = np.arange(0, Nsamp, 1)
        cols = self.px        
        #cols = np.tile(cols, N_scan) 
        self.P = csc_matrix((ones, (rows, cols)), shape = (Nsamp, Npix))
        self.PT = csc_matrix(self.P.T)
        
    def get_F(self):
        Nsamp, Nbaseline, Nperbaselines = self.Nsamp, self.Nbaseline, self.Nperbaselines
        Nperbaselines_cum = np.zeros(Nbaseline + 1)
        Nperbaselines_cum = np.cumsum(Nperbaselines)
        
        ones = np.ones(Nsamp)
        rows = np.arange(0, Nsamp, 1)
        cols = np.zeros(Nsamp)
        
        for i in range(Nbaseline):
            start = Nperbaselines_cum[i]
            end = Nperbaselines_cum[i + 1]
            cols[start:end] = np.tile(i, Nperbaselines[i + 1])
        self.F = csc_matrix((ones, (rows, cols)), shape = (Nsamp, Nbaseline))
        self.FT = csc_matrix(self.F.T)
        
        """
        ones = np.ones(Nsamp)
        rows = np.arange(0, Nsamp, 1)
        cols = np.zeros(Nsamp)
        for i in range(Nbaseline):
            cols[i * Nperbaseline:(i+1) * Nperbaseline] = np.tile(i, Nperbaseline)
        self.F = csc_matrix((ones, (rows, cols)), shape = (Nsamp, Nbaseline))
        """
    
    def get_Cn_inv(self):
        Nsamp, Nscans, cumlen = self.Nsamp, self.Nscans, self.tod_cumlen
        C_n_inv = np.ones(Nsamp)
        #Nperscan = int(Nsamp / Nscans)
        for i in range(Nscans):
            start = cumlen[i]
            end = cumlen[i + 1]
            #print(start, end, Nsamp, 1 / self.sigma0[i])
            C_n_inv[start:end] *= 1 / self.sigma0[i]
        #print(np.sum(np.isinf(1 / self.sigma0)))
        self.C_n_inv = diags(C_n_inv)
    
    def get_PCP_inv(self):
        P, PT, C_n_inv = self.P, self.PT, self.C_n_inv
        
        #PCP_inv = PT.dot(C_n_inv)        
        #PCP_inv = PCP_inv.dot(P) + diags(eps * np.ones(self.Npix))
        
        PCP_inv = PT.dot(C_n_inv)        
        PCP_inv = sparse_dot_mkl.dot_product_mkl(PCP_inv, P) + diags(eps * np.ones(self.Npix))
           
        self.PCP_inv = diags(1 / PCP_inv.diagonal(), format = "csc")
   
    def get_FT_C_P_PCP(self):
        FT, C_n_inv, P, PCP_inv = self.FT, self.C_n_inv, self.P, self.PCP_inv
        #FT_C = FT.dot(C_n_inv)        
        #FT_C_P = FT_C.dot(P)
        #self.FT_C_P_PCP = FT_C_P.dot(PCP_inv)
    
        FT_C = FT.dot(C_n_inv)        
        FT_C_P = sparse_dot_mkl.dot_product_mkl(FT_C, P)
        self.FT_C_P_PCP = FT_C_P.dot(PCP_inv)
    
    def get_PT_C(self):
        PT, C_n_inv = self.PT, self.C_n_inv
        self.PT_C = PT.dot(C_n_inv)
        #self.PT_C = sparse_dot_mkl.dot_product_mkl(PT, C_n_inv)
        
    
    def get_FT_C(self):
        FT, C_n_inv = self.FT, self.C_n_inv
        self.FT_C = FT.dot(C_n_inv)        
        #self.FT_C = sparse_dot_mkl.dot_product_mkl(FT, C_n_inv)        
    
        
    def Ax(self, a):
        FT_C_F, FT_C_P_PCP, PT_C_F = self.FT_C_F, self.FT_C_P_PCP, self.PT_C_F
      
        #temp0 = FT_C_F.dot(a)
        #temp1 = PT_C_F.dot(a)
        #temp2 = FT_C_P_PCP.dot(temp1)
        
        temp0 = sparse_dot_mkl.dot_product_mkl(FT_C_F, a)
        temp1 = sparse_dot_mkl.dot_product_mkl(PT_C_F, a)
        temp2 = sparse_dot_mkl.dot_product_mkl(FT_C_P_PCP, temp1)
        
        return temp0 - temp2
        
    def b(self, x):        
        FT_C, FT_C_P_PCP, PT_C = self.FT_C, self.FT_C_P_PCP, self.PT_C
        
        #temp0 = FT_C.dot(x)
        #temp1 = PT_C.dot(x)
        #temp2 = FT_C_P_PCP.dot(temp1)
        
        temp0 = sparse_dot_mkl.dot_product_mkl(FT_C, x)
        temp1 = sparse_dot_mkl.dot_product_mkl(PT_C, x)
        temp2 = sparse_dot_mkl.dot_product_mkl(FT_C_P_PCP, temp1)
        
        return temp0 - temp2
    
    def get_destriped_map(self):
        
        
        P, PT, C_n_inv, F, tod, Nsamp, eps, PCP_inv = self.P, self.PT, self.C_n_inv, self.F, self.tod, self.Nsamp, self.eps, self.PCP_inv
        
        self.get_FT_C()
        self.get_FT_C_P_PCP()
        self.get_PT_C()
        
        #self.FT_C_F = self.FT_C.dot(F)
        #self.PT_C_F = self.PT_C.dot(F)
        self.FT_C_F = sparse_dot_mkl.dot_product_mkl(self.FT_C, F)
        self.PT_C_F = sparse_dot_mkl.dot_product_mkl(self.PT_C, F)
        
        Ax = linalg.LinearOperator((self.Nbaseline, self.Nbaseline) , matvec = self.Ax)
        b  = self.b(tod)
        
        self.a, info = linalg.cg(Ax, b)
        
        m = PCP_inv.dot(PT).dot(C_n_inv).dot(tod - F.dot(self.a))
        self.m = m.reshape(self.Nside, self.Nside)

    def get_bin_averaged_map(self, highpass = False, cut = 0.1):
        #self.get_P()
        P, PT, tod, eps = self.P, self.PT, self.tod, self.eps
        
        if highpass:
            sos = signal.butter(10, cut, "highpass", output = "sos")
            tod = signal.sosfilt(sos, tod)
            
        #A = P.transpose().dot(P) + diags(eps * np.ones(self.Npix))
        #b = P.transpose().dot(tod)
        
        A = sparse_dot_mkl.dot_product_mkl(PT, P) + diags(eps * np.ones(self.Npix))
        b = sparse_dot_mkl.dot_product_mkl(PT, tod)
        self.m = linalg.spsolve(A, b).reshape(self.Nside, self.Nside)
    
    def get_noise_weighted_map(self, highpass = False, cut = 0.1):
        #self.get_P()
        #self.get_Cn_inv()
        
        P, PT, C_n_inv, tod, Nsamp, eps, PCP_inv = self.P, self.PT, self.C_n_inv, self.tod, self.Nsamp, self.eps, self.PCP_inv 
        
        if highpass:
            sos = signal.butter(10, cut, "highpass", output = "sos")
            tod = signal.sosfilt(sos, tod)
            
        #PCP_inv = P.transpose().dot(C_n_inv)        
        #PCP_inv = PCP_inv.dot(P) + diags(eps * np.ones(self.Npix))
        #PCP_inv = diags(1 / PCP_inv.diagonal())
        
        #self.m = PCP_inv.dot(P.transpose()).dot(C_n_inv).dot(tod).reshape(self.Nside, self.Nside)
        self.m = PCP_inv.dot(PT).dot(C_n_inv).dot(tod).reshape(self.Nside, self.Nside)
    
    def get_hits(self):
        #self.get_P()
        P, Nside, Nsamp = self.P, self.Nside, self.Nsamp
        ones = np.ones(Nsamp)
        self.hits = P.transpose().dot(ones).reshape(Nside, Nside)
        
    def get_rms(self):
        #self.get_P()
        #self.get_Cn_inv()
        P, C_n_inv, Nsamp, Nside, eps, PCP_inv = self.P, self.C_n_inv, self.Nsamp, self.Nside, self.eps, self.PCP_inv
        
        #PCP_inv = P.transpose().dot(C_n_inv)        
        #PCP_inv = PCP_inv.dot(P) + diags(eps * np.ones(self.Npix))
        #PCP_inv = diags(1 / PCP_inv.diagonal())
        self.rms = PCP_inv.diagonal().reshape(Nside, Nside)
        
    def load_cube(self):
        """
        Read the simulated datacube into memory.
        """
        cube = np.load(self.cube_filename)
        cubeshape = cube.shape

        cube *= 1e-6    # Normalization of cube by input value
        cube *= 1000     # Normalization of cube by input value
        print("MAX CUBE:", np.nanmax(cube))
        cube = cube.reshape(cubeshape[0], cubeshape[1], 4, 1024)  # Flatten the x/y dims, and split the frequency (depth) dim in 4 sidebands.
        cube = cube.reshape(cubeshape[0], cubeshape[1], 4, 64, 16)
        cube = np.mean(cube, axis = 4)     # Averaging over 16 frequency channels
        print(cube.shape)
        cube = cube.transpose(2, 3, 0, 1)
        print(cube.shape)

        self.cube = cube[nsb, nfreq, :, :]



In [30]:
if __name__ == "__main__":
    nsb     = 2
    nfreq   = 15
    nfeed   = 17
    #infile  = "/mn/stornext/d16/cmbco/comap/nils/COMAP_general/data/level2/Ka/sim/dynamicTsys/co6/co6_001498708.h5"
    #infile  = "/mn/stornext/d16/cmbco/comap/nils/COMAP_general/data/level2/Ka/sim/dynamicTsys/co6/co6_001527505.h5"
    infile  = "/mn/stornext/d16/cmbco/comap/nils/COMAP_general/data/level2/Ka/sim/dynamicTsys/co6/co6_001512706.h5"
    datapath    = "/mn/stornext/d16/cmbco/comap/nils/COMAP_general/data/level2/Ka/sim/dynamicTsys/co6/"
    #datapath  = "/mn/stornext/d16/cmbco/comap/nils/COMAP_general/data/level2/Ka/sim/poly/zeroth/co6/"
    
    N_baseline = 10
    highcut = 0.02 #Hz 
    #highcut = 25 #Hz 
    eps     = 1e-8


    t = time.time()
    destr = Destriper(datapath, highcut, eps)
    destr.run(nsb, nfreq)
    
    print(destr.tod.shape)
    destr.get_destriped_map()

    m_destr = destr.m 

    destr.get_noise_weighted_map(highpass = False, cut = 0.02)
    m_weighted = destr.m 
    
    destr.get_bin_averaged_map(highpass = False, cut = 0.02)
    m_avg = destr.m 

    destr.get_hits()
    hits = destr.hits
    
    destr.get_rms()
    rms = destr.rms
    print("Total time elapsed:", time.time() - t, "sec")
    """
    #A = destr.A.toarray()
    A = destr.A
    M = A.copy()
    M = M.diagonal()
    M = 1/M
    M = diags(M)
    A = M.dot(A).toarray()
    fig5, ax5 = plt.subplots(figsize = (8, 8))
    im5 = ax5.imshow(A, vmin = -1.5, vmax = 1.5)
    divider5 = make_axes_locatable(ax5)
    cax5 = divider5.append_axes("right", size="5%", pad=0.05)
    cbar = fig5.colorbar(im5, ax=ax5, cax = cax5)#, extend = "both")
    fig5.tight_layout()
    """
    
    

Loading data:
Number of scans: 27
Loading time: 13.128302574157715 sec
Get pixel index:
Get pixel time: 1.9729089736938477 sec
Get pointing matrix:
Get pointing matrix time: 0.1524827480316162 sec
Get F:
Get F time: 0.19176173210144043 sec
Get C_n_inv:
Get C_n_inv time: 0.022083759307861328 sec
Get PCP_inv:
Get PCP_inv time: 0.3124237060546875 sec
(9272752,)
Total time elapsed: 17.63679552078247 sec


In [3]:
    def dummy(arr):
        d = arr.copy()
        print(arr.shape)
        for i in len(arr):
            d[i] = np.sqrt(np.sqrt(np.sum(arr)))
        return d

    pool = multiproc.Pool(5)
    array = np.random.randint(0, 9, (11, 10))
    print(type(array), type(dummy))
    result = pool.map(dummy, array)
    print(result)

(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)(10,)(10,)


(10,)
<class 'numpy.ndarray'> <class 'function'>


TypeError: 'int' object is not iterable

In [23]:
    m_destr     = np.ma.masked_where(hits < 1, m_destr)
    m_weighted  = np.ma.masked_where(hits < 1, m_weighted)
    m_avg       = np.ma.masked_where(hits < 1, m_avg)
    hits        = np.ma.masked_where(hits < 1, hits)
    rms         = np.ma.masked_where(hits < 1, rms)
    
    cmap_name = "CMRmap"
    cmap = copy.copy(plt.get_cmap(cmap_name))
    cmap.set_bad("0.8", 1)
    

In [24]:

    fig0, ax0 = plt.subplots(figsize = (8, 7))
    
    
    im0 = ax0.imshow(m_destr.T * 1e6, cmap = cmap, vmin = -5000, vmax = 5000)
    im0.set_rasterized(True)
    divider0 = make_axes_locatable(ax0)
    cax0 = divider0.append_axes("right", size="5%", pad=0.05)
    cbar0 = fig0.colorbar(im0, ax=ax0, cax = cax0)
    ax0.set_title("Destriper")
    cbar0.set_label(r"$\mu K$")
    fig0.tight_layout()
    #plt.savefig("Destriper.png")

   

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: figure size must be positive finite not [0. 0.]

ValueError: figure size must be positive finite not [0. 0.]

In [25]:
 
    fig1, ax1 = plt.subplots(figsize = (8, 7))
    
    im1 = ax1.imshow(m_weighted.T  * 1e6, cmap = cmap, vmin = -5000, vmax = 5000)
    im1.set_rasterized(True)

    im1.set_rasterized(True)
    divider1 = make_axes_locatable(ax1)
    cax1 = divider1.append_axes("right", size="5%", pad=0.05)
    cbar1 = fig1.colorbar(im1, ax=ax1, cax = cax1)
    cbar1.set_label(r"$\mu K$")
    ax1.set_title("Noise Weighted")
    fig1.tight_layout()
    
    #plt.savefig("NoiseWeighted.png")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: figure size must be positive finite not [0. 0.]

ValueError: figure size must be positive finite not [0. 0.]

In [26]:
    fig2, ax2 = plt.subplots(figsize = (8, 7))
    
    im2 = ax2.imshow(m_avg.T  * 1e6, cmap = cmap, vmin = -5000, vmax = 5000)
    im2.set_rasterized(True)
    im2.set_rasterized(True)
    divider2 = make_axes_locatable(ax2)
    cax2 = divider2.append_axes("right", size="5%", pad=0.05)
    cbar2 = fig2.colorbar(im2, ax=ax2, cax = cax2)
    ax2.set_title("Bin Averaged")
    cbar2.set_label(r"$\mu K$")
    fig2.tight_layout()

    #plt.savefig("BinAveraged.png")
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: figure size must be positive finite not [0. 0.]

ValueError: figure size must be positive finite not [0. 0.]

In [27]:
    fig3, ax3 = plt.subplots(figsize = (8, 7))
    
    im3 = ax3.imshow(hits.T, cmap = cmap)
    im3.set_rasterized(True)
    divider3 = make_axes_locatable(ax3)
    cax3 = divider3.append_axes("right", size="5%", pad=0.05)
    fig3.colorbar(im3, ax=ax3, cax = cax3)
    fig3.tight_layout()
    


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: figure size must be positive finite not [0. 0.]

ValueError: figure size must be positive finite not [0. 0.]

In [28]:
    fig4, ax4 = plt.subplots(figsize = (8, 7))
    im4 = ax4.imshow(rms.T, cmap = cmap, vmin = 0, vmax = 0.001)
    im4.set_rasterized(True)
    divider4 = make_axes_locatable(ax4)
    cax4 = divider4.append_axes("right", size="5%", pad=0.05)
    fig4.colorbar(im4, ax=ax4, cax = cax4)
    fig4.tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: figure size must be positive finite not [0. 0.]

ValueError: figure size must be positive finite not [0. 0.]

In [15]:
    diff_destr_avg = (m_destr - m_avg)  * 1e6
    diff_destr_weighted = (m_destr - m_weighted)  * 1e6
    diff_weighted_avg = (m_weighted - m_avg)  * 1e6

In [16]:
    fig6, ax6 = plt.subplots(figsize = (8, 7))
    im6 = ax6.imshow(diff_destr_avg.T, cmap = cmap, vmin = -5000, vmax = 5000)
    im6.set_rasterized(True)
    divider6 = make_axes_locatable(ax6)
    cax6 = divider6.append_axes("right", size="5%", pad=0.05)
    cbar6 = fig6.colorbar(im6, ax=ax6, cax = cax6)
    ax6.set_title(r"$m_\mathrm{destriper} - m_\mathrm{bin avg}$")
    cbar6.set_label(r"$\mu K$")
    fig6.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
    fig7, ax7 = plt.subplots(figsize = (8, 7))
    im7 = ax7.imshow(diff_destr_weighted.T, cmap = cmap, vmin = -5000, vmax = 5000)
    im7.set_rasterized(True)
    divider7 = make_axes_locatable(ax7)
    cax7 = divider7.append_axes("right", size="5%", pad=0.05)
    cbar7 = fig7.colorbar(im7, ax=ax7, cax = cax7)
    ax7.set_title(r"$m_\mathrm{destriper} - m_\mathrm{noise weighed}$")
    cbar7.set_label(r"$\mu K$")
    fig7.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
    fig8, ax8 = plt.subplots(figsize = (8, 7))
    im8 = ax8.imshow(diff_weighted_avg.T, cmap = cmap, vmin = -500, vmax = 500)
    im8.set_rasterized(True)
    divider8 = make_axes_locatable(ax8)
    cax8 = divider8.append_axes("right", size="5%", pad=0.05)
    cbar8 = fig8.colorbar(im8, ax=ax8, cax = cax8)
    ax8.set_title(r"$m_\mathrm{noise weighted} - m_\mathrm{bin avg}$")
    cbar8.set_label(r"$\mu K$")
    fig8.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
    destr.load_cube()
    cube = destr.cube
    cube = np.ma.masked_where(hits < 1, cube)
    

MAX CUBE: 0.13597469378256236
(120, 120, 4, 64)
(4, 64, 120, 120)


In [20]:
    fig9, ax9 = plt.subplots(figsize = (8, 7))
    im9 = ax9.imshow(cube.T * 1e6, cmap = cmap, vmin = -2000, vmax = 2000)
    im9.set_rasterized(True)
    divider9 = make_axes_locatable(ax9)
    cax9 = divider9.append_axes("right", size="5%", pad=0.05)
    cbar9 = fig9.colorbar(im9, ax=ax9, cax = cax9)
    ax9.set_title(r"$m_\mathrm{sim}$")
    cbar9.set_label(r"$\mu K$")
    fig9.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [96]:
    res_destr = m_destr - cube
    res_weighted = m_weighted - cube
    res_avg = m_avg - cube


In [97]:
    fig10, ax10 = plt.subplots(figsize = (8, 7))
    im10 = ax10.imshow(res_destr.T * 1e6, cmap = cmap, vmin = -10000.0, vmax = 10000.0)
    im10.set_rasterized(True)
    divider10 = make_axes_locatable(ax10)
    cax10 = divider10.append_axes("right", size="5%", pad=0.05)
    cbar10 = fig10.colorbar(im10, ax=ax10, cax = cax10)
    ax10.set_title(r"$m_\mathrm{destriped} - m_\mathrm{sim}$")
    cbar10.set_label(r"$\mu K$")
    fig10.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [98]:
    fig11, ax11 = plt.subplots(figsize = (8, 7))
    im11 = ax11.imshow(res_weighted.T * 1e6, cmap = cmap, vmin = -10000.0, vmax = 10000.0)
    im11.set_rasterized(True)
    divider11 = make_axes_locatable(ax11)
    cax11 = divider11.append_axes("right", size="5%", pad=0.05)
    cbar11 = fig11.colorbar(im11, ax=ax11, cax = cax11)
    ax11.set_title(r"$m_\mathrm{destriped} - m_\mathrm{sim}$")
    cbar11.set_label(r"$\mu K$")
    fig11.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
    fig12, ax12 = plt.subplots(figsize = (8, 7))
    im12 = ax12.imshow(res_avg.T * 1e6, cmap = cmap, vmin = -10000.0, vmax = 10000.0)
    im12.set_rasterized(True)
    divider12 = make_axes_locatable(ax12)
    cax12 = divider12.append_axes("right", size="5%", pad=0.05)
    cbar12 = fig12.colorbar(im12, ax=ax12, cax = cax12)
    ax12.set_title(r"$m_\mathrm{destriped} - m_\mathrm{sim}$")
    cbar12.set_label(r"$\mu K$")
    fig12.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
#chi2_destr = (np.nansum((res_destr / rms) ** 2) - 120 ** 2) / np.sqrt(2 * 120 ** 2)
#chi2_weighted = (np.nansum((res_weighted / rms) ** 2) - 120 ** 2) / np.sqrt(2 * 120 ** 2)
#chi2_avg = (np.nansum((res_avg / rms) ** 2) - 120 ** 2) / np.sqrt(2 * 120 ** 2)

chi2_destr = np.nansum((res_destr / rms) ** 2) 
chi2_weighted = np.nansum((res_weighted / rms) ** 2) 
chi2_avg = np.nansum((res_avg / rms) ** 2)

print(chi2_destr)
print(chi2_weighted)
print(chi2_avg)

551985.8619393678
530221.865489179
537126.3959101299


In [ ]:
    sbs     = np.arange(4)
    freqs   = np.arange(64)
    feeds   = np.arange(18)
    
    datapath    = "/mn/stornext/d16/cmbco/comap/nils/COMAP_general/data/level2/Ka/sim/dynamicTsys/co6/"
    
    N_baseline = 10
    highcut = 0.02 #Hz 
    eps     = 1e-8
    
    m_destr_tot = np.zeros((4, 64, 120, 120))
    m_weighted_tot = np.zeros_like(m_destr_tot)
    m_avg_tot = np.zeros_like(m_destr_tot)
    m_rms_tot = np.zeros_like(m_destr_tot)
    m_nhit_tot = np.zeros_like(m_destr_tot)
    m_cube_tot = np.zeros_like(m_destr_tot)
    
    for nsb in sbs:
        print("Sideband:", nsb)
        for nfreq in freqs:
            destr = Destriper(nsb, nfreq, 10, datapath, highcut, eps)
            destr.run()

            destr.get_destriped_map()
            m_destr_tot[nsb, nfreq, ...] = destr.m 

            destr.get_noise_weighted_map()
            m_weighted_tot[nsb, nfreq, ...] = destr.m 

            destr.get_bin_averaged_map()
            m_avg_tot[nsb, nfreq, ...] = destr.m 

            destr.get_rms()
            m_rms_tot[nsb, nfreq, ...] = destr.rms

            destr.get_hits()
            m_nhit_tot[nsb, nfreq, ...] = destr.hits
            
            destr.load_cube()
            m_cube_tot[nsb, nfreq, ...] = destr.cube
    """
    A = destr.A.toarray()
    fig5, ax5 = plt.subplots(figsize = (8, 8))
    im5 = ax5.imshow(A)
    divider5 = make_axes_locatable(ax5)
    cax5 = divider5.append_axes("right", size="5%", pad=0.05)
    fig5.colorbar(im5, ax=ax5, cax = cax5)
    fig5.tight_layout()
    """
    
    

Sideband: 0
Loading data:
Number of scans: 27
242 [   0 2500 2500 2500 2500 2500 2500 2500 2500  150 2500 2500 2500 2500
 2500 2500 2500 2500  151 2500 2500 2500 2500 2500 2500 2500 2500  103
 2500 2500 2500 2500 2500 2500 2500 2500   75 2500 2500 2500 2500 2500
 2500 2500 2500   51 2500 2500 2500 2500 2500 2500 2500 2500  101 2500
 2500 2500 2500 2500 2500 2500 2500 2500 2500 2500 2500 2500 2500 2500
 2500   77 2500 2500 2500 2500 2500 2500 2500 2500  150 2500 2500 2500
 2500 2500 2500 2500 2500  775 2500 2500 2500 2500 2500 2500 2500 2500
  202 2500 2500 2500 2500 2500 2500 2500 2500  201 2500 2500 2500 2500
 2500 2500 2500 2500  750 2500 2500 2500 2500 2500 2500 2500 2500  103
 2500 2500 2500 2500 2500 2500 2500 2500  126 2500 2500 2500 2500 2500
 2500 2500 2500  100 2500 2500 2500 2500 2500 2500 2500 2500   77 2500
 2500 2500 2500 2500 2500 2500 2500  125 2500 2500 2500 2500 2500 2500
 2500 2500  228 2500 2500 2500 2500 2500 2500 2500 2500  178 2500 2500
 2500 2500 2500 2500 2500 2

In [ ]:
    m_destr_tot     = np.ma.masked_where(m_nhit_tot < 1, m_destr_tot)
    m_weighted_tot  = np.ma.masked_where(m_nhit_tot < 1, m_weighted_tot)
    m_avg_tot       = np.ma.masked_where(m_nhit_tot < 1, m_avg_tot)
    m_rms_tot       = np.ma.masked_where(m_nhit_tot < 1, m_rms_tot)
    m_cube_tot      = np.ma.masked_where(m_nhit_tot < 1, m_cube_tot)


In [ ]:
    fig13, ax13 = plt.subplots(figsize = (8, 7))
    im13 = ax13.imshow(m_destr_tot[2, 15,...].T * 1e6, cmap = cmap, vmin = -10000.0, vmax = 10000.0)
    im13.set_rasterized(True)
    divider13 = make_axes_locatable(ax13)
    cax13 = divider13.append_axes("right", size="5%", pad=0.05)
    cbar13 = fig13.colorbar(im13, ax=ax13, cax = cax13)
    ax13.set_title(r"$m_\mathrm{destriped} - m_\mathrm{sim}$")
    cbar13.set_label(r"$\mu K$")
    fig13.tight_layout()